# Aviary Competition Scoreing Procedure 2023

## Imports

Make sure to install pandas in your python enviroment

In [ ]:
import pandas as pd

# Import data

In [ ]:
sends = pd.read_csv('sample_data.csv')

In [ ]:
sends = sends.rename(columns={'Email Address' : 'email',
                              'Full Name' : 'name',
                              'Category' : 'category',
                              'Anchor?' : 'anchor',
                              'Colour?': 'colour',
                              'Number of Attempts' : 'num_attempts',
                              'Belayer (Full Name)' : 'belayer',
                              'Witness #2 (Full Name)' : 'witness'})


In [ ]:
sends.head()

## Data Cleaning

In [ ]:
# all strings to lower case and strip of leading and lagging spaces
sends['email'] = sends['email'].str.lower().str.strip()
sends['name'] = sends['name'].str.lower().str.strip()
sends['belayer'] = sends['belayer'].str.lower().str.strip()
sends['witness'] = sends['witness'].str.lower().str.strip()

In [ ]:
# Fix route mismatch and commonly entered wrong routes
sends.loc[sends['colour'] == 'Hand Crack', 'anchor'] = 2  # The hand crack is on anchor 2

sends.loc[(sends['anchor'] == 1) & (sends['colour'] == 'Red'), 'anchor'] = 2 # 1 Red -> 2 Red

sends.loc[(sends['anchor'] == 5) & (sends['colour'] == 'Green'), 'colour'] = 'Lime' # 5 Green -> 5 Lime

sends.loc[(sends['anchor'] == 11) & (sends['colour'] == 'Pink'), 'anchor'] = 12 # 11 Pink -> 12 Pink


### Fill in missing values for name and category
We made the name and category questions optional because user feedback indicated there were too many questions to fill out for each response. 
For responses without a name and category, we expect there will exist one repsone using the same email to contain a name and category and will use that value for all other responses without a name and category. 


In [ ]:
## There must be a better way to do this 🤷 

sends_filled = pd.concat([sends['email'], sends.groupby('email').fillna(method='ffill')], axis=1)
sends_filled = pd.concat([sends['email'], sends_filled.groupby('email').fillna(method='bfill')], axis=1)

## Check Assumptions
The scoring section assumes:
1. there are no duplicate entries (ie the same route is submitted twice)
2. people have entered the same name for all their entries (ie a individual has submitted all their entires under the same name)
3. people dont have the same name
4. people only entered one category

We should probably make sure this is true.

The challenge would be that solving these errors would likely have to be done manually.

In [ ]:
# 1 - Identify anchor/colour duplicates, will have to confirm manually which ones to keep/drop

sends_filled[sends_filled.duplicated(subset=['name', 'email', 'anchor', 'colour'], keep=False)]

In [ ]:
# 2/3 - check list of names to see if any name variations can be manually identified
sends_filled[['name','email']].drop_duplicates().sort_values(by=['name', 'email'])

In [ ]:
# emails with more than one name would give us hints to if someone mistyped their name
email_counts = sends_filled[['name','email']].groupby(by='email').nunique()
email_counts[email_counts['name'] != 1]

In [ ]:
# names with more than one email would give us hints to if two people share the same name (or someone mistyped their email)
name_counts = sends_filled[['name','email']].groupby(by='name').nunique()
name_counts[name_counts['email'] != 1]

In [ ]:
name_counts = sends_filled[['name','category']].groupby(by='name').nunique()
name_counts[name_counts['category'] != 1]

We can now proceed with these assumptions

## Bind Points to Sends

Points per route

In [ ]:
points_per_route = pd.read_csv('points_per_route.csv')
points_per_route.sort_values(by=['anchor', 'colour']).head()

Points for number of attemps

In [ ]:
points_per_attempt = pd.read_csv('points_per_attempt.csv')
points_per_attempt.head()

Bind points (by route) to sends

In [ ]:
sends_points = pd.merge(sends_filled, points_per_route, on=['anchor', 'colour'], how='left')

In [ ]:
sends_points = pd.merge(sends_points, points_per_attempt, on='num_attempts', how='left')

In [ ]:
sends_points.head()

In [ ]:
# Check to see if anyone claimed to climb a route I didnt know about 
sends_points[sends_points['route_points'].isna()]

### Caluclate total points

In [ ]:
sends_points['points'] = sends_points['route_points'] * sends_points['attempts_points']
sends_points.head()

# Scores

### Recreational - men

In [ ]:
# subset the points df
rec_men = sends_points.loc[sends_points['category'] == 'Recreational - men']

# group sends by individual and sum points
rec_men[['name','points']].groupby(by=['name']).sum().sort_values(by='points', ascending=False)

### Open - men

In [ ]:
# subset the points df
open_men = sends_points.loc[sends_points['category'] == 'Open - men']

# group sends by individual and sum points
open_men[['name','points']].groupby(by=['name']).sum().sort_values(by='points', ascending=False)

### Recreational - anyone but men

In [ ]:
# subset the points df
rec_abm = sends_points.loc[sends_points['category'] == 'Recreational - anyone but men']

# group sends by individual and sum points
rec_abm[['name','points']].groupby(by=['name']).sum().sort_values(by='points', ascending=False)

### Open - anyone but men

In [ ]:
# subset the points df
open_abm = sends_points.loc[sends_points['category'] == 'Open - anyone but men']

# group sends by individual and sum points
open_abm[['name','points']].groupby(by=['name']).sum().sort_values(by='points', ascending=False)

### Top Belayer

In [ ]:
sends_points['belayer'].value_counts()

### Most (total) attempts

on sent routes

In [ ]:
sends_points[['name', 'attempts']].groupby(by='name').sum().sort_values(by='attempts', ascending=False).head(n=5)

### Most routes climbed

Just counts the number of submission by an individual

In [ ]:
sends_points.groupby(by=['name']).size().sort_values(ascending=False)